In [1]:
import os
from pathlib import Path

from types import SimpleNamespace
from pprint import pprint

import torch

from cellseg_exp import experiment
from cellseg_utils import prepare_data, get_str_timestamp

No display found. Using non-interactive Agg backend.


In [26]:
draw = True
test = True

if test:
    run_clear_ml = False
    out_dir = 'out/test'
    shuffle = False
    ratio_train = 0.0
    ratio_val = 0.0
    # ratio_train = 0.6
    # ratio_val = 0.2
    # images_num = 20
    images_num = 10
    max_epochs = 0
else:
    run_clear_ml = True
    out_dir = 'out/LF1'
    shuffle = True
    ratio_train = 0.6
    ratio_val = 0.2
    images_num = None
    max_epochs = 600

root_dir = Path('datasets/Cells_2.0_for_Ivan/masked_MSC')
dir01 = root_dir / 'pics 2024-20240807T031703Z-001' / 'pics 2024'
dir02 = root_dir / 'pics 2024-20240807T031703Z-002' / 'pics 2024'
lf_dir = dir01 / 'LF1'
resize_coef = 1

dataset_dir = lf_dir
exp_class_dict = {'+2024-05-05-LF1-p6-sl2': 6,
                  '+2024-05-06-LF1-p12': 12,
                  '+2024-05-06-LF1p9-sl2': 9,
                  '+2024-05-07-LF1p15': 15,
                  '+2024-05-08-LF1p18sl2': 18,
                  '+2024-05-31-LF1-p22': 22
                  }

params = dict(
    model_name='Unet',
    model_load_fp=None,
    # model_load_fp='out/MSC_filtered_mc/Unet_timm-efficientnet-b0_20240425_160139/best_model.pth',
    # model_load_fp=os.path.join('out', 'MSC', 'Unet_timm-efficientnet-b6_20240417_144310', 'best_model.pth'),
    # model_load_fp=Path('out/LF1') / 'Unet_timm-efficientnet-b6_20241012_154141' / 'best_model.pth',
    model_load_full=True,

    dataset_dir=dataset_dir,
    exp_class_dict=exp_class_dict,
    ratio_train=ratio_train,
    ratio_val=ratio_val,
    
    square_a=256,
    border=10,
    classes_num=1+1,
    channels=4,
    num_workers=1,
    batch_size=3,
    bce_weight=0.1,

    ENCODER='timm-efficientnet-b0',  # 'resnet101',  # 'efficientnet-b2',  # 'timm-efficientnet-b8',  # 'efficientnet-b0'
    ENCODER_WEIGHTS='imagenet',
    ACTIVATION='sigmoid',  # could be None for logits or 'softmax2d' for multiclass segmentation
    DEVICE='cuda' if torch.cuda.is_available() else 'cpu',

    max_epochs=max_epochs,
    lr_first=1e-2,
    lr_last=1e-4,
)

In [27]:
params = SimpleNamespace(**params)
classes = list(exp_class_dict.values())
classes.sort()
train_dataset, valid_dataset, test_dataset = prepare_data(params, classes, images_num=images_num, shuffle=shuffle,
                                                          resize_coef=resize_coef)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.05it/s]


In [28]:
from cellseg_utils import (
    split_image,
    unsplit_image
)

In [29]:
image = test_dataset.images[0]
mask = test_dataset.masks[0]
shadow = test_dataset.shadows[0]
info = test_dataset.info[0]
print(image.shape)
print(mask.shape)
print(shadow.shape)
print(info.keys())


(3072, 4064, 3)
(3072, 4064, 2)
(3072, 4064, 1)
dict_keys(['cls', 'idx', 'mask_fp', 'r_fp', 'g_fp', 'b_fp', 'p_fp'])


In [30]:
    # images = list()
    # masks = list()
    # squares_info = list()
    # for image, mask, shadow in tqdm(list(zip(test_dataset.images, test_dataset.masks, test_dataset.shadows))):
    #     assert image.shape[:-1] == mask.shape[:-1] == shadow.shape[:-1]

    #     mask = self._prepare_mask(mask / 255)

    #     # split
    #     _, img_sq_list = split_image(img, self.full_size,
    #                                  self.squares, self.border)
    #     _, mask_sq_list = split_image(mask, self.full_size,
    #                                   self.squares, self.border)
    #     if self.add_shadow_to_img:
    #         shadow = self.read_shadow(fp_data)
    #         _, shadow_sq_list = split_image(shadow, self.full_size,
    #                       self.squares, self.border)
    #     else:
    #         shadow_sq_list = [None] * len(img_sq_list)

    #     for img_sq, msk_sq, shd_sq, sq in zip(img_sq_list,
    #                                   mask_sq_list, shadow_sq_list,
    #                                   self.squares):

    #         img_sq, msk_sq = self.aug(img, mask, shadow=shd_sq)
    #         img_sq = img_sq.transpose(2, 0, 1)
    #         msk_sq = msk_sq.transpose(2, 0, 1)

    #         images.append(img_sq.astype(np.uint8))
    #         masks.append(msk_sq.astype(np.uint8))
    #         # cur_cq = sq.copy()
    #         # cur_cq['fp'] = fp
    #         squares_info.append(sq)

    # # images = np.stack(images, axis=0)
    # # masks = np.stack(masks, axis=0)
    # return images, masks, squares_info

In [31]:
assert image.shape[:-1] == mask.shape[:-1] == shadow.shape[:-1]

In [32]:
test_dataset.full_size

(4096, 3072)

In [ ]:
# split
_, img_sq_list = split_image(image, test_dataset.full_size, test_dataset.squares, test_dataset.border)
_, mask_sq_list = split_image(mask, test_dataset.full_size, test_dataset.squares, test_dataset.border)

In [ ]:
img_sq_list

In [ ]:
if self.add_shadow_to_img:
    shadow = self.read_shadow(fp_data)
    _, shadow_sq_list = split_image(shadow, self.full_size,
                  self.squares, self.border)
else:
    shadow_sq_list = [None] * len(img_sq_list)

for img_sq, msk_sq, shd_sq, sq in zip(img_sq_list,
                              mask_sq_list, shadow_sq_list,
                              self.squares):

    img_sq, msk_sq = self.aug(img, mask, shadow=shd_sq)
    img_sq = img_sq.transpose(2, 0, 1)
    msk_sq = msk_sq.transpose(2, 0, 1)

    images.append(img_sq.astype(np.uint8))
    masks.append(msk_sq.astype(np.uint8))
    # cur_cq = sq.copy()
    # cur_cq['fp'] = fp
    squares_info.append(sq)